In [1]:
import xarray as xr
import s3fs
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [11]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['ssp245'], table_id=['day'], variable_id=['od550aer','clt','hus','va','tas'], member_id=['r1i1p1f1'])
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,va,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
1,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,tas,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
2,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,hus,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
3,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,clt,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108


In [7]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [9]:
dataset_list = list(dset_dict.keys())

In [10]:
dset = dset_dict[dataset_list[0]]
dset = dset.sel(member_id='r1i1p1f1',time=slice("2000-01-01", "2014-12-31"))
dset

<xarray.Dataset>
Dimensions:    (lat: 96, lon: 144, time: 5475, bnds: 2, plev: 8)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) object 2000-01-01 12:00:00 ... 2014-12-31 12:00:00
    member_id  <U8 'r1i1p1f1'
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(5475, 1), meta=np.ndarray>
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    clt        (time, lat, lon) float32 dask.array<chunksize=(818, 96, 144), meta=np.ndarray>
    hus        (time, plev, lat, lon) float32 dask.array<chunksize=(30, 8, 96, 144), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1453, 96, 144), meta=np.ndarray>
    va         (time, plev, lat, lon) float32 dask.array<chunksize=(54, 8, 96, 144), meta=np.ndarray>
Attributes: (12/50)
    Conventions:               CF-1.7 CMIP-6.2
    run_variant:               N/A
    sub_experiment_id:         none
    parent_source_id:          NorESM2-LM
    title:                     NorESM2-LM output prepared for CMIP6
    experiment:                all-forcing simulation of the recent past
    ...                        ...
    sub_experiment:            none
    parent_activity_id:        CMIP
    table_info:                Creation Date:(24 July 2019) MD5:08e314340b9dd...
    nominal_resolution:        250 km
    intake_esm_varname:        clt\nhus\ntas\nva
    intake_esm_dataset_key:    CMIP.NCC.NorESM2-LM.historical.day.gn

In [9]:
# import AOD historical
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0",client_kwargs=dict(endpoint_url="https://rgw.met.no"))
s3.ls('escience2022/Remy/')

s3path = "s3://escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_20000101-20091231.nc"
esacci = xr.open_dataset(s3.open(s3path))

In [10]:
esacci

<xarray.Dataset>
Dimensions:     (time: 3650, bnds: 2, lat: 96, lon: 144)
Coordinates:
  * time        (time) object 2000-01-01 12:00:00 ... 2009-12-31 12:00:00
  * lat         (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon         (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    wavelength  float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds   (time, bnds) object ...
    lat_bnds    (lat, bnds) float64 ...
    lon_bnds    (lon, bnds) float64 ...
    od550aer    (time, lat, lon) float32 ...
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 1600-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     430335.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               od550aer
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/3ce70ca9-d644-452d-8cec-b385c7a6a89d

In [15]:
s3.ls('escience2022/Remy/')

['escience2022/Remy/.keep',
 'escience2022/Remy/.wget-20221102193645.sh.status',
 'escience2022/Remy/.wget-20221102193729.sh.status',
 'escience2022/Remy/.wget-20221102193735.sh.status',
 'escience2022/Remy/.wget-20221102193742.sh.status',
 'escience2022/Remy/Era5',
 'escience2022/Remy/dl_era5_hus.py',
 'escience2022/Remy/dl_era5_wind.py',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19500101-19591231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19600101-19691231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19700101-19791231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19800101-19891231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19900101-19991231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_20000101-20091231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_20100101-20141231.nc',
 'escie

In [20]:
# import AOD historical
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0",client_kwargs=dict(endpoint_url="https://rgw.met.no"))
s3.ls('escience2022/Remy/')

s3path = list(['escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20150101-20201231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20210101-20301231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20410101-20501231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20510101-20601231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20610101-20701231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20710101-20801231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20810101-20901231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20910101-21001231.nc'])

sopenlist=[s3.open(ss) for ss in s3path]

ssp245 = xr.open_mfdataset(sopenlist)